In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/working/my_awesome_model'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/working/my_awesome_model/checkpoint-3000/model.safetensors
/kaggle/working/my_awesome_model/checkpoint-3000/config.json
/kaggle/working/my_awesome_model/checkpoint-3000/trainer_state.json
/kaggle/working/my_awesome_model/checkpoint-3000/rng_state.pth
/kaggle/working/my_awesome_model/checkpoint-3000/tokenizer.json
/kaggle/working/my_awesome_model/checkpoint-3000/vocab.txt
/kaggle/working/my_awesome_model/checkpoint-3000/special_tokens_map.json
/kaggle/working/my_awesome_model/checkpoint-3000/tokenizer_config.json
/kaggle/working/my_awesome_model/checkpoint-3000/optimizer.pt
/kaggle/working/my_awesome_model/checkpoint-3000/scheduler.pt
/kaggle/working/my_awesome_model/checkpoint-3000/training_args.bin
/kaggle/working/my_awesome_model/checkpoint-2000/model.safetensors
/kaggle/working/my_awesome_model/checkpoint-2000/config.json
/kaggle/working/my_awesome_model/checkpoint-2000/trainer_state.json
/kaggle/working/my_awesome_model/checkpoint-2000/rng_state.pth
/kaggle/working/my_aweso

In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset
data=load_dataset("/kaggle/input/allocine-french-movie-reviews")

In [24]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
})

In [4]:
# Renommer les colonnes et supprimer les colonnes inutiles
data = data.map(lambda x: {"text": x["review"], "label": x["polarity"]})
data = data.remove_columns(['Unnamed: 0', 'film-url', 'review', 'polarity'])


In [5]:
#data=load_dataset('imdb')

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
def preprocess_function(examples):
    
     return tokenizer(examples["text"], truncation=True)

   

In [8]:
tokenized_imdb = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
# Définir le modèle
id2label = {0: "negative", 1: "positive"}
label2id = {"negative": 0, "positive": 1}

In [11]:
print(tokenized_imdb["train"].features)


{'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


In [12]:
!pip install transformers datasets evaluate accelerate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
import evaluate

accuracy = evaluate.load("accuracy")

In [14]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [18]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,  # Sauvegarde toutes les 1000 étapes
    eval_steps=1000,  # Évaluation toutes les 1000 étapes
    load_best_model_at_end=True,
    save_total_limit=3,  # Conserver uniquement les 3 derniers checkpoints
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


trainer.train(resume_from_checkpoint=True)


/opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss,Validation Loss,Accuracy
4000,0.212600,0.205355,0.933500
5000,0.195400,0.181482,0.939600
6000,0.184100,0.211350,0.936650
7000,0.177900,0.168735,0.945500
8000,0.174500,0.213482,0.944050
9000,0.166800,0.161974,0.947200
10000,0.177200,0.161752,0.948500
11000,0.134500,0.174505,0.951350
12000,0.140700,0.176011,0.948000
13000,0.125400,0.176126,0.952400


TrainOutput(global_step=20000, training_loss=0.1291647247314453, metrics={'train_runtime': 18506.9733, 'train_samples_per_second': 17.291, 'train_steps_per_second': 1.081, 'total_flos': 7.550016133545792e+16, 'train_loss': 0.1291647247314453, 'epoch': 2.0})

In [19]:
trainer.save_model("my_awesome_model/final_model")

In [22]:
!zip -r my_awesome_model.zip my_awesome_model/final_model/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


updating: my_awesome_model/final_model/ (stored 0%)
updating: my_awesome_model/final_model/model.safetensors (deflated 7%)
updating: my_awesome_model/final_model/config.json (deflated 51%)
updating: my_awesome_model/final_model/tokenizer.json (deflated 71%)
updating: my_awesome_model/final_model/vocab.txt (deflated 53%)
updating: my_awesome_model/final_model/special_tokens_map.json (deflated 42%)
updating: my_awesome_model/final_model/tokenizer_config.json (deflated 76%)
updating: my_awesome_model/final_model/training_args.bin (deflated 51%)


In [25]:
# Préparer les données de test
test_dataset = tokenized_imdb["test"]

# Évaluer le modèle sur les données de test
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [26]:
print(test_results)

{'eval_loss': 0.15533015131950378, 'eval_accuracy': 0.94955, 'eval_runtime': 297.7234, 'eval_samples_per_second': 67.176, 'eval_steps_per_second': 4.199, 'epoch': 2.0}


In [29]:
# Enregistrer le modèle évalué
trainer.save_model("my_awesome_model/final_models")  # Enregistrement du modèle
tokenizer.save_pretrained("my_awesome_model/final_models")  # Enregistrement du tokenizer

('my_awesome_model/final_models/tokenizer_config.json',
 'my_awesome_model/final_models/special_tokens_map.json',
 'my_awesome_model/final_models/vocab.txt',
 'my_awesome_model/final_models/added_tokens.json',
 'my_awesome_model/final_models/tokenizer.json')

In [53]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Charger le modèle et le tokenizer depuis le répertoire sauvegardé
model = AutoModelForSequenceClassification.from_pretrained("my_awesome_model/checkpoint-20000")
tokenizer = AutoTokenizer.from_pretrained("my_awesome_model/checkpoint-20000")


In [60]:
# Exemple de texte à prédire
text = "le film m'a permet d'avoir des millions de vue sur linedkin"

# Tokenizer le texte
inputs = tokenizer(text, return_tensors="pt")


In [61]:
import torch
# Faire une prédiction
with torch.no_grad():  # Désactiver la calcul de gradients pour l'inférence
    outputs = model(**inputs)

# Obtenir les logits
logits = outputs.logits

# Convertir les logits en prédictions (par exemple, pour une classification)
predictions = logits.argmax(dim=-1)
print(f"Prédiction : {predictions.item()}")


Prédiction : 1
